In [92]:
pip install atproto -q
pip install ace_tools_open -q


SyntaxError: invalid syntax (2946105168.py, line 1)

In [93]:
from atproto import FirehoseSubscribeReposClient, parse_subscribe_repos_message

# Initialize Firehose client
client = FirehoseSubscribeReposClient()

MAX_MESSAGES = 1  # Limit messages for testing
message_count = 0  # Counter to track received messages

def on_message_handler(message) -> None:
    """Handler that stops after receiving MAX_MESSAGES"""
    global message_count
    if message_count >= MAX_MESSAGES:
        print("✅ Reached message limit. Stopping Firehose.")
        client.stop()
        return

    try:
        parsed_msg = parse_subscribe_repos_message(message)
        print("🔹 Received Message:", parsed_msg)

        message_count += 1

    except Exception as e:
        print(f"❌ Error processing message: {e}")

# Start the Firehose stream
client.start(on_message_handler)


🔹 Received Message: blobs=[] blocks=b':\xa2eroots\x81\xd8*X%\x00\x01q\x12 d\xd7\x08ma\x13@>\xd9\x9c$\xc2\xc1&\xb7\x1cv\xf0\xd9\r&(\x1f^\xed\x83\xf4\x1b\xc3J\rmgversion\x01\xb9\x02\x01q\x12 \xbb\x88F\xfcQ\x1b\x88S\x1a\xa9\xc3\xf3\x0f\xa7\x18\xc9\xca\x1c\x14\\W\xeb\x13K\x03C\xb2\x9e\xc6\xeb\x10\\\xa2ae\x82\xa4akX app.bsky.feed.like/3ldpu4sneve2uap\x00at\xd8*X%\x00\x01q\x12 2\xee\x9f\x1a)\x18\xa3]\xa95z=~F\'\xe0\x13})\x9cGwM\xba\xff\x9d\xd7\xcc\xba\xf7\xe5\xa9av\xd8*X%\x00\x01q\x12 nA^\xb91\x8d\xe9e\x90\xf7d\xdc\x15\xf1\xf9\x1e\xb6\x17O\xb7\x074Xq\x01\x94\x0c\x07\x05\xcbq/\xa4akKiufgojbix2tap\x15at\xd8*X%\x00\x01q\x12 \xf6\x10nz\x05F\xf4\x85"\xe5\x80\x0e\x03\xa8\xd5\xe4LH\xfe\xae\xc1\xbe\xb4C\x1d\xf3\x0e&\xb2\xe9g`av\xd8*X%\x00\x01q\x12 \x14\xc4\x1a\x86_r\xc7\xb5w\x86\xce?m\xcd\x98\xcf\xf2\x87\x97\x84\x85~\x0c\x0c5MDu8\x84\xe1\xbbal\xd8*X%\x00\x01q\x12 B>\xb4\x14\xd1\xb6G\r\xc1\xfaW\xd5\x8cW\x03\xdf#\xc0\xd88C\xfe_\x0c\x99PB\xcd\'\x0e\xc3]S\x01q\x12 \xf6\x10nz\x05F\xf4\x85"\xe5\x80\x0e\x0

In [94]:
from atproto import CAR, models, FirehoseSubscribeReposClient, parse_subscribe_repos_message
import json
import threading

client = FirehoseSubscribeReposClient()

MAX_MESSAGES = 50  # Limit messages for testing
message_count = 0  # Counter
firehose_data = []  # List to store structured messages

def stop_firehose():
    """Stops the Firehose listener after MAX_MESSAGES messages."""
    print("✅ Reached message limit. Stopping Firehose.")
    client.stop()

def on_message_handler(message) -> None:
    """Handler that parses Firehose messages and extracts key data"""
    global message_count
    if message_count >= MAX_MESSAGES:
        stop_firehose()
        return

    try:
        commit = parse_subscribe_repos_message(message)
        
        if not isinstance(commit, models.ComAtprotoSyncSubscribeRepos.Commit):
            print("⚠️ Skipping non-commit message.")
            return

        if not commit.blocks:
            print("⚠️ No blocks found in message.")
            return

        car = CAR.from_bytes(commit.blocks)

        structured_data = {
            "repo": commit.repo,
            "timestamp": commit.time,
            "rev": commit.rev,
            "seq": commit.seq,
            "ops": []
        }

        for op in commit.ops:
            if op.action != "create" or not op.cid:
                continue

            uri = f"at://{commit.repo}/{op.path}"
            record_raw_data = car.blocks.get(op.cid)
            if not record_raw_data:
                continue

            record = models.get_or_create(record_raw_data, strict=False)
            
            structured_data["ops"].append({
                "action": op.action,
                "path": op.path,
                "cid": str(op.cid),
                "record": record.__dict__,  # Convert record to dictionary
            })

        firehose_data.append(structured_data)  # Store structured data
        
        message_count += 1

        # If MAX_MESSAGES reached, stop Firehose in a separate thread
        if message_count >= MAX_MESSAGES:
            threading.Thread(target=stop_firehose).start()

        print(f"✅ Processed Message {message_count}/{MAX_MESSAGES} from {commit.repo}")

    except Exception as e:
        print(f"❌ Error processing message: {e}")

# Start the Firehose stream
client.start(on_message_handler)

# Display structured messages after collection
firehose_data[:10]  # Show first 3 parsed messages


✅ Processed Message 1/50 from did:plc:w4wpyhjhga4hsqas2ttfhkl3
✅ Processed Message 2/50 from did:plc:a3xxkroznluc6xgdxgsqbbds
✅ Processed Message 3/50 from did:plc:v7fo4mtnn5sp74sj7dmxahxy
✅ Processed Message 4/50 from did:plc:z7jtkk5rzn77glcijjhdgcas
✅ Processed Message 5/50 from did:plc:iovstgs47gt4255eqtw5c5bh
✅ Processed Message 6/50 from did:plc:h736nwqwqwo2fzydfruta5x6
✅ Processed Message 7/50 from did:plc:wr4aacygfoy75ysw6crmjxjm
✅ Processed Message 8/50 from did:plc:hu2zabe3v5q2x6tubfup25it
✅ Processed Message 9/50 from did:plc:jglo5gxqlgvmqxvpupa4scyv
✅ Processed Message 10/50 from did:plc:7djmrvewxgxpgwbejfm2dmq3
✅ Processed Message 11/50 from did:plc:ivj2bcujiplux5cdjp6s4ye4
✅ Processed Message 12/50 from did:plc:tqvfx6jilnmy2v46y55hntrl
✅ Processed Message 13/50 from did:plc:awaxsie6anubiiy47amtia3m
✅ Processed Message 14/50 from did:plc:oolduqewawcj574jjfow2qhb
✅ Processed Message 15/50 from did:plc:jx6wtgdwga6o5reejakenqax
✅ Processed Message 16/50 from did:plc:oj2tfjysfe

[{'repo': 'did:plc:w4wpyhjhga4hsqas2ttfhkl3',
  'timestamp': '2025-03-03T19:23:58.615Z',
  'rev': '3ljiogl6hdq2e',
  'seq': 5949964286,
  'ops': [{'action': 'create',
    'path': 'app.bsky.feed.like/3ljiogl5sti2e',
    'cid': 'bafyreiggcs3hdhumo3v6ehjxwrgngm5ugnxbexqnbdh2l754vysyp32ro4',
    'record': {'created_at': '2025-03-03T19:23:58.498719+00:00',
     'subject': Main(cid='bafyreie7hgjpudwhmfuci3j25qpnawka2cmazirq3foqftby2qhrmyw26m', uri='at://did:plc:jaqih6pcxnqgd2rfvpeoc2a3/app.bsky.feed.post/3ljay3zwdok2z', py_type='com.atproto.repo.strongRef'),
     'py_type': 'app.bsky.feed.like'}}]},
 {'repo': 'did:plc:a3xxkroznluc6xgdxgsqbbds',
  'timestamp': '2025-03-03T19:23:58.615Z',
  'rev': '3ljiogl7nsk2l',
  'seq': 5949964287,
  'ops': [{'action': 'create',
    'path': 'app.bsky.feed.like/3ljiogl7gxs2l',
    'cid': 'bafyreidl5iwmfdwcs2xx3vojadwef5p4d7gnkmaj7ft6pvs5dlsdicjrxm',
    'record': {'created_at': '2025-03-03T19:23:57.815Z',
     'subject': Main(cid='bafyreigzhk7fueorcocqyid6jw

In [95]:
import sqlite3
import json
import base64


def convert_to_dict(obj):
    """ Recursively convert non-serializable objects into JSON-compatible format. """
    if isinstance(obj, dict):
        return {key: convert_to_dict(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_dict(item) for item in obj]
    elif isinstance(obj, bytes):  # 🛠️ Convert bytes to a base64 string
        return base64.b64encode(obj).decode('utf-8')
    elif hasattr(obj, "__dict__"):  # Convert custom objects with attributes
        return {key: convert_to_dict(value) for key, value in vars(obj).items()}
    else:
        return obj  # Return basic types as-is

# Connect to SQLite
conn = sqlite3.connect("firehose.db")
cur = conn.cursor()

# 🔥 Drop and Recreate Tables with More Structured Columns
cur.execute("DROP TABLE IF EXISTS posts")
cur.execute("""
    CREATE TABLE posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repo TEXT,
        timestamp TEXT,
        seq INTEGER,
        text TEXT,
        langs TEXT,
        parent_cid TEXT,
        parent_uri TEXT,
        post_uri TEXT,  --!!! might not work !!!!
        cid TEXT,
        root_cid TEXT,
        root_uri TEXT
    )
""")

cur.execute("DROP TABLE IF EXISTS likes")
cur.execute("""
    CREATE TABLE likes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repo TEXT,
        timestamp TEXT,
        seq INTEGER,
        liked_post_cid TEXT,
        liked_post_uri TEXT
    )
""")

cur.execute("DROP TABLE IF EXISTS reposts")
cur.execute("""
    CREATE TABLE reposts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repo TEXT,
        timestamp TEXT,
        seq INTEGER,
        reposted_post_cid TEXT,
        reposted_post_uri TEXT
    )
""")

cur.execute("DROP TABLE IF EXISTS follows")
cur.execute("""
    CREATE TABLE follows (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repo TEXT,
        timestamp TEXT,
        seq INTEGER,
        followed_user TEXT
    )
""")

conn.commit()

# ✅ Function to Construct Post URI from repo and cid
def construct_post_uri(repo, cid):
    """ Constructs a Bluesky post URI from repo (DID) and CID. """
    return f"at://{repo}/app.bsky.feed.post/{cid}" if repo and cid else None

# Insert Records into the Appropriate Tables
for message in firehose_data:
    for op in message["ops"]:
        event_type = op["path"].split("/")[0]  # Extract event type

        # 🛠️ Ensure `record` is a dictionary (convert JSON string if needed)
        record_data = json.loads(op["record"]) if isinstance(op["record"], str) else op["record"]

        # ✅ Convert objects inside `record_data` into JSON-compatible format
        record_data_serializable = convert_to_dict(record_data)

        # Extract common fields
        repo = message["repo"]
        timestamp = message["timestamp"]
        seq = message["seq"]
        cid = op.get("cid", None)  # ✅ Extract CID from Firehose event
        post_uri = construct_post_uri(repo, cid)  # ✅ Construct Post URI

        if event_type == "app.bsky.feed.post":
            text = record_data_serializable.get("text", None)
            langs = ",".join(record_data_serializable.get("langs", [])) if record_data_serializable.get("langs") else None

            # 🛠️ Fix: Ensure `reply` data is safely extracted
            reply_data = record_data_serializable.get("reply", {}) or {}

            parent_cid = reply_data.get("parent", {}).get("cid", None)
            parent_uri = reply_data.get("parent", {}).get("uri", None)
            root_cid = reply_data.get("root", {}).get("cid", None)
            root_uri = reply_data.get("root", {}).get("uri", None)

            # Insert structured data into `posts`
            cur.execute("""
                INSERT INTO posts (repo, timestamp, seq, text, langs, parent_cid, parent_uri, post_uri, cid, root_cid, root_uri)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (repo, timestamp, seq, text, langs, parent_cid, parent_uri, post_uri, cid, root_cid, root_uri))

        elif event_type == "app.bsky.feed.like":
            subject = record_data_serializable.get("subject", {})
            liked_post_cid = subject.get("cid", None) if isinstance(subject, dict) else None
            liked_post_uri = subject.get("uri", None) if isinstance(subject, dict) else None

            # Insert structured data into `likes`
            cur.execute("""
                INSERT INTO likes (repo, timestamp, seq, liked_post_cid, liked_post_uri)
                VALUES (?, ?, ?, ?, ?)
            """, (repo, timestamp, seq, liked_post_cid, liked_post_uri))

        elif event_type == "app.bsky.feed.repost":
            subject = record_data_serializable.get("subject", {})
            reposted_post_cid = subject.get("cid", None) if isinstance(subject, dict) else None
            reposted_post_uri = subject.get("uri", None) if isinstance(subject, dict) else None

            # Insert structured data into `reposts`
            cur.execute("""
                INSERT INTO reposts (repo, timestamp, seq, reposted_post_cid, reposted_post_uri)
                VALUES (?, ?, ?, ?, ?)
            """, (repo, timestamp, seq, reposted_post_cid, reposted_post_uri))

        elif event_type == "app.bsky.graph.follow":
            followed_user = record_data_serializable.get("subject", None)  # Followed user is a direct string

            # Insert structured data into `follows`
            cur.execute("""
                INSERT INTO follows (repo, timestamp, seq, followed_user)
                VALUES (?, ?, ?, ?)
            """, (repo, timestamp, seq, followed_user))

conn.commit()
conn.close()

print("Firehose data successfully saved into separate tables with structured columns.")


Firehose data successfully saved into separate tables with structured columns.


In [96]:
import sqlite3
import pandas as pd
import ace_tools_open as tools

# Connect to SQLite
conn = sqlite3.connect("firehose.db")
cur = conn.cursor()

# Function to display tables
def display_table(table_name):
    print(f"\n Displaying records from {table_name} table:")
    query = f"SELECT * FROM {table_name} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    tools.display_dataframe_to_user(name=f"{table_name} Table", dataframe=df)

# List of tables to inspect
tables = ["posts", "likes", "reposts", "follows"]

# Display each table
for table in tables:
    display_table(table)

conn.close()



 Displaying records from posts table:
posts Table



 Displaying records from likes table:
likes Table



 Displaying records from reposts table:
reposts Table



 Displaying records from follows table:
follows Table


In [108]:
def generate_bluesky_link(uri):
    """ Converts an AT Protocol URI into a Bluesky web link. """
    if uri.startswith("at://"):
        parts = uri.split("/")
        if len(parts) == 5:
            did, collection, record_id = parts[2], parts[3], parts[4]
            return f"https://bsky.app/profile/{did}/post/{record_id}"
    return None

# Example Usage
uri = "at://did:plc:q2grkt2qns6lwbyxyxirfbf7/app.bsky.feed.post/bafyreieii2rxcugqivsz23rhsyojpwlavt4z27ds7uluyigiucuyyfobva"
link = generate_bluesky_link(uri)
print("Bluesky Post Link:", link)


Bluesky Post Link: https://bsky.app/profile/did:plc:q2grkt2qns6lwbyxyxirfbf7/post/bafyreieii2rxcugqivsz23rhsyojpwlavt4z27ds7uluyigiucuyyfobva


## UP UNTIL HERE CORRECTO !!!!

In [115]:
"""def construct_original_post_uri(repo, cid):
    """ Generates a Bluesky URI for standalone posts using the DID and CID. """
    if repo and cid:
        return f"at://{repo}/app.bsky.feed.post/{cid}"
    return None

repo = "did:plc:q2grkt2qns6lwbyxyxirfbf7"
cid = "bafyreieii2rxcugqivsz23rhsyojpwlavt4z27ds7uluyigiucuyyfobva"  # Correct identifier

uri = construct_original_post_uri(repo, cid)
print("🔗 Reconstructed Post URI:", uri)



SyntaxError: invalid syntax (566651408.py, line 2)

In [105]:
import sqlite3
import pandas as pd
import ace_tools_open as tools

# Connect to SQLite
conn = sqlite3.connect("firehose.db")

# Fetch a few standalone posts
df_check = pd.read_sql_query("""
    SELECT post_uri FROM posts
    WHERE parent_cid IS NULL
    LIMIT 20;
""", conn)

conn.close()

# Display the post_uri values
print(df_check["post_uri"].tolist())  # Print a list of `post_uri` values


['at://did:plc:tqvfx6jilnmy2v46y55hntrl/app.bsky.feed.post/bafyreiaodbfq5l7bfynotxytkbs2rwyyxudwxa4vr7bcm3iucro2nx2b34', 'at://did:plc:oj2tfjysfe7quvz5av456p3t/app.bsky.feed.post/bafyreidv6poo6ru6fsonsmjo5zfqvfejadh53v3epajqohmsznn3wm3vme', 'at://did:plc:q2grkt2qns6lwbyxyxirfbf7/app.bsky.feed.post/bafyreieii2rxcugqivsz23rhsyojpwlavt4z27ds7uluyigiucuyyfobva']


In [109]:
import requests

# 🔹 Replace this with an actual Bluesky handle if available
user_did = "did:plc:tqvfx6jilnmy2v46y55hntrl"
post_cid = "bafyreieii2rxcugqivsz23rhsyojpwlavt4z27ds7uluyigiucuyyfobva"

# 🔹 Bluesky API endpoint to fetch posts (you might need authentication)
url = f"https://bsky.app/xrpc/com.atproto.repo.listRecords?repo={user_did}&collection=app.bsky.feed.post"

response = requests.get(url)
if response.status_code == 200:
    posts = response.json()
    print(posts)  # Check if your post is in this list
else:
    print(f"⚠️ Failed to fetch data: {response.status_code}")


⚠️ Failed to fetch data: 404


In [113]:
import requests

# 🔹 Your Bluesky credentials
BLUESKY_USERNAME = "UN"
BLUESKY_PASSWORD = "PW"

# 🔹 Authenticate and get access token
session = requests.Session()
auth_url = "https://bsky.social/xrpc/com.atproto.server.createSession"
auth_payload = {"identifier": BLUESKY_USERNAME, "password": BLUESKY_PASSWORD}
auth_response = session.post(auth_url, json=auth_payload)

if auth_response.status_code == 200:
    access_token = auth_response.json().get("accessJwt")
    print("✅ Authentication successful!")

    # 🔹 Fetch posts
    headers = {"Authorization": f"Bearer {access_token}"}
    user_did = "did:plc:tqvfx6jilnmy2v46y55hntrl"
    url = f"https://bsky.social/xrpc/com.atproto.repo.listRecords?repo={user_did}&collection=app.bsky.feed.post"

    response = session.get(url, headers=headers)
    
    if response.status_code == 200:
        posts = response.json()
        print("🔍 Full API Response:")
        print(posts)  # Print full API response to check structure

    else:
        print(f"⚠️ Failed to fetch posts: {response.status_code} - {response.text}")

else:
    print(f"❌ Authentication failed: {auth_response.status_code} - {auth_response.text}")


✅ Authentication successful!
🔍 Full API Response:
{'records': [{'uri': 'at://did:plc:tqvfx6jilnmy2v46y55hntrl/app.bsky.feed.post/3ljipmbns722z', 'cid': 'bafyreihuicbngpgmpx2uaaownhkggvncr2ca2j6vsdpf3hrnpp3p3wx344', 'value': {'text': 'Excellent interview! Thank you @acyn.bsky.social and @meidastouch.com. #ElbowsUp', '$type': 'app.bsky.feed.post', 'langs': ['en'], 'reply': {'root': {'cid': 'bafyreieyjetp34w5he2whucviynrizfinu22gr4e2q3lsoggn6hlodej3i', 'uri': 'at://did:plc:iu4j537hox5huj4bwnwgub4z/app.bsky.feed.post/3ljikjcrfbk2v'}, 'parent': {'cid': 'bafyreiatolvb7jfdaceggom4k7jigkbz6ixx65cz4wvqvy7zjazanbnvjy', 'uri': 'at://did:plc:iu4j537hox5huj4bwnwgub4z/app.bsky.feed.post/3ljikjcrpzc2v'}}, 'facets': [{'$type': 'app.bsky.richtext.facet', 'index': {'byteEnd': 48, 'byteStart': 31}, 'features': [{'did': 'did:plc:iu4j537hox5huj4bwnwgub4z', '$type': 'app.bsky.richtext.facet#mention'}]}, {'$type': 'app.bsky.richtext.facet', 'index': {'byteEnd': 69, 'byteStart': 53}, 'features': [{'did': 'did

In [114]:
for post in posts.get("records", []):
    print(f"🔹 Post CID: {post.get('cid')}")
    print(f"🔹 rkey: {post.get('rkey')}")  # This is what we need for URLs!


🔹 Post CID: bafyreihuicbngpgmpx2uaaownhkggvncr2ca2j6vsdpf3hrnpp3p3wx344
🔹 rkey: None
🔹 Post CID: bafyreigi7pmm6q6wfukwgqorjbnwf3te6xzu5zztexz3c2rejopomrusxa
🔹 rkey: None
🔹 Post CID: bafyreibr4yd5tvbbcmav2hqofgh6nttvnrdeivywlobjuld7nygrp3ijkq
🔹 rkey: None
🔹 Post CID: bafyreie7tsqjmr74keckdk7bvuzffryvnfjj7y7juxupkxz55isyndta24
🔹 rkey: None
🔹 Post CID: bafyreifpohlh3xumcmp7dlhauvgzjhzaholzcpz7o22gbecomel6a2bqim
🔹 rkey: None
🔹 Post CID: bafyreicmi3c4siby53hwqb37iggcqzlpfjum4su6ezjlxb6v3elcygetrm
🔹 rkey: None
🔹 Post CID: bafyreie2wx6thfugxfttqgbudtgdvxwwmhcmh23dhgtok7yc33a4gyylfu
🔹 rkey: None
🔹 Post CID: bafyreihwmtdomrvv5gpr5pvtkawkkmacupftltksfiulqkch6k6w7fcbrm
🔹 rkey: None
🔹 Post CID: bafyreihuu5jvyw2hi2jm34aprj5pvyvt6zu2f3c6yhio6llf62ff4gachu
🔹 rkey: None
🔹 Post CID: bafyreiaxfmvdpcezem5vjpoydl4yxbyrfrgnros37geimpze6kx4yjffty
🔹 rkey: None
🔹 Post CID: bafyreidicv7myu57tlwibna3hui2euhrdbeofove53qd3nw6xzw72ldds4
🔹 rkey: None
🔹 Post CID: bafyreihqvbxtlw7df6ha74moom7evh5bn36jxuiiwuq6gc3dxejd